In [34]:
#!pip install PyPDF2 yfinance openai==1.21.0
!pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
  Attempting uninstall: openai
    Found existing installation: openai 1.21.0
    Uninstalling openai-1.21.0:
      Successfully uninstalled openai-1.21.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires aiohttp<4.0.0,>=3.9.1, but you have aiohttp 3.7.4 which is incompatible.
instructor 0.5.2 requires rich<14.0.0,>=13.7.0, but you have rich 10.2.0 which is incompatible.
langchain 0.1.13 requires aiohttp<4.0.0,>=3.8.3, but you have aiohttp 3.7.4 which is incompatible.
langchain-community 0.0.29 requires aiohttp<4.0.0,>=3.8.3, but you have aiohttp 3.7.4 which is incompatible.
marker 2.1.5 requires requests==2.25.1, but you have requests 2.31.0 which is incompatible.


In [2]:
from openai import OpenAI
import openai
import os



api_key = "sk-bwq6F1VjDiipTnAmSjNQbQ5w7fhl9TGhUZpejYOiLNT3BlbkFJ2yhicBsobmXIa-jWus2TAVBrr52D9YxojRyuKAv4cA" 

client = openai.OpenAI(api_key=api_key)

assistant_id = "asst_1kszslvJyXy7PGsNwwdUnIrV" #@param {type:"string"}

my_assistant = client.beta.assistants.retrieve(assistant_id)
print("Assistant Details:", my_assistant)

Assistant Details: Assistant(id='asst_1kszslvJyXy7PGsNwwdUnIrV', created_at=1724914082, description=None, file_ids=[], instructions='You are an virtual assistant for restaurant . You specialize in giving customer service', metadata={}, model='gpt-4o-mini', name='Resturant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], top_p=1.0, temperature=1.0, response_format={'type': 'text'})


UPDATE

In [ ]:
##@markdown Enter the Assistant ID to update:
#assistant_id = "asst_1kszslvJyXy7PGsNwwdUnIrV" #@param {type:"string"}
#
##@markdown Enter the new name for the Assistant:
#new_name = "The Cool Robot Assistant" #@param {type:"string"}
#
##@markdown Enter the new description for the Assistant:
#new_description = "Hi How Are You?" #@param {type:"string"}
#
#updated_assistant = client.beta.assistants.update(
#    assistant_id=assistant_id,
#    name=new_name,
#    description=new_description,
#    model="gpt-4-turbo",
#)
#print(updated_assistant)

In [3]:
#@title **OpenAI Assistant Configuration for Real Estate**
#@markdown This script configures and interacts with an OpenAI assistant specialized in real estate. Please provide the required details below.

import time
from openai import OpenAI
import openai
import os

from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
            
            
            
#@markdown Enter the Assistant ID to update:
assistant_id = "asst_1kszslvJyXy7PGsNwwdUnIrV" #@param {type:"string"}


# Create a new thread
thread = client.beta.threads.create()
print(f"Thread created with ID: {thread.id}")

#@markdown Enter your query for the real estate assistant:
user_query = "Is your food delicious?" #@param {type:"string"}

# Create a user message in the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_query
)


# Retrieve the assistant's response message
response_message = None
messages = client.beta.threads.messages.list(thread_id=thread.id).data

for message in messages:
    if message.role == "assistant":
        response_message = message
        break

if response_message:
    print(f"Assistant response: {response_message.content}")
else:
    print("No assistant response found.")


Thread created with ID: thread_vnBqDWkkWXUFJiRnQz1dYN38


AttributeError: 'Runs' object has no attribute 'create_and_poll'

In [13]:
import openai
import time
import yfinance as yf


api_key = "sk-bwq6F1VjDiipTnAmSjNQbQ5w7fhl9TGhUZpejYOiLNT3BlbkFJ2yhicBsobmXIa-jWus2TAVBrr52D9YxojRyuKAv4cA" 

client = openai.OpenAI(api_key=api_key)

#assistant_id = "asst_1kszslvJyXy7PGsNwwdUnIrV" #@param {type:"string"}

tools_object = {
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}

def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price




assistant = client.beta.assistants.create(
  instructions="You are a financial analyst assistant for a trading. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo",
  tools=[tools_object]
    
)

# Step 1: Create an Assistant
#assistant = assistant_id

# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you please provide me stock price of Apple?"
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Mervin Praison."
)

print(run.model_dump_json(indent=4))

while True:
    # Wait for 5 seconds
    time.sleep(2)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": str(output)
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

{
    "id": "run_kN9WCuiTozvLohHv2GnaHfwn",
    "assistant_id": "asst_Bg8uOud9uqp0AIdQUFRbZylC",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1729416035,
    "expires_at": 1729416635,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Mervin Praison.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-3.5-turbo",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_439ku6yIOwGNqQyhe7rFbLVp",
    "tools": [
        {
            "function": {
                "name": "get_stock_price",
                "description": "Retrieve the latest closing price of a stock using its ticker symbol",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The ticker symbol of t

In [ ]:
api_key = "sk-bwq6F1VjDiipTnAmSjNQbQ5w7fhl9TGhUZpejYOiLNT3BlbkFJ2yhicBsobmXIa-jWus2TAVBrr52D9YxojRyuKAv4cA" 

In [22]:
import openai
import time
import yfinance as yf
import json

# OpenAI API key setup
api_key = "sk-bwq6F1VjDiipTnAmSjNQbQ5w7fhl9TGhUZpejYOiLNT3BlbkFJ2yhicBsobmXIa-jWus2TAVBrr52D9YxojRyuKAv4cA" 

# Initialize OpenAI client and ensure it's using the v2 Assistants API
client = openai.OpenAI(
    api_key=api_key,
    default_headers={"OpenAI-Beta": "assistants=v2"}
)

# Function to retrieve stock price using yfinance
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

# Define the tool as a custom function in v2 API structure
tools_object = {
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}

# Step 1: Create an assistant using OpenAI with only the custom function tool
assistant = client.beta.assistants.create(
    instructions="You are a financial analyst assistant for a trading platform. Use the provided functions to answer questions.",
    model="gpt-4o",  # You can also use gpt-3.5-turbo
    tools=[tools_object]  # Pass the custom function tool inside a list
)

# Step 2: Create a Thread (No assistant_id needed here in v2)
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you please provide me stock price of Apple?"
)

# Step 4: Run the Assistant
# IMPORTANT: Now passing the assistant_id when running the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,  # Add assistant_id here
    instructions="Please address the user as Mervin Praison."  # Instructions for the assistant
)

print(run.model_dump_json(indent=4))

while True:
    # Wait for 2 seconds
    time.sleep(2)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []

        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                
                # Convert output to string as required by the API
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": str(output)  # Convert float to string
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)


BadRequestError: Error code: 400 - {'error': {'message': "The requested model 'gpt-4o' cannot be used with the Assistants API in v1. Follow the migration guide to upgrade to v2: https://platform.openai.com/docs/assistants/migration.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'unsupported_model'}}

In [35]:
assistant = client.beta.assistants.create(
  instructions="You are a financial bot. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo",
  tools=[
    {
      "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
      }
    }
  ]
)


thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the stock price of apple?",
)



run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
 
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_stock_price":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "57.0"
    })
 
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
  
  
  

AttributeError: 'Runs' object has no attribute 'create_and_poll'